## **Mapeando datos**

➡️ Apuntar

🗒️ **Observaciones:**

⚠️ **Alertas:**

### **1. Importamos las respectivas librerías**

In [1]:
import pandas as pd
import zipfile
from pathlib import Path
import shutil
import folium
import sqlite3
import matplotlib.colors as mcolors

### **2. Descomprimimos los archivos correspondientes**

➡️ Creamos una variable path que nos ayude a indentificar donde estan todos los archivos zip.

In [2]:
raw_path = '/home/npalaciosv/Catedra/Geoanalitycs/src/Data/Raw/'
silver_path = '/home/npalaciosv/Catedra/Geoanalitycs/src/Data/Silver/'

data_files = Path(raw_path)

➡️ Necesitamos recorrer la carpeta ya que habra mas de un archivo zip, luego vamos a descomprimirlos, almacenarlos en un dataframe y juntarlos todos. Usamos la libreria Path.

In [3]:
for file in data_files.rglob('*'):
    new_name = file.name.replace(" ","").replace("_","")
    new_path = file.with_name(new_name)
    file.rename(new_path)
    with zipfile.ZipFile(f'{raw_path+file.name}','r') as extraction:
        extraction.extractall(silver_path)

➡️ Eliminamos los archivos que tienen un formato diferente a csv.

🗒️ **Observaciones:** La librería path tambien es útil en este caso.

In [4]:
silver_files = Path(silver_path)
for file in silver_files.iterdir():
    # Verificar si es un archivo y no termina en '.csv'
    if file.is_file() and file.suffix != '.csv':
        # Eliminar el archivo
        file.unlink()
        print(f'Eliminado: {file}')

Eliminado: /home/npalaciosv/Catedra/Geoanalitycs/src/Data/Silver/2020_SemII.dta
Eliminado: /home/npalaciosv/Catedra/Geoanalitycs/src/Data/Silver/2019_SemII.dta
Eliminado: /home/npalaciosv/Catedra/Geoanalitycs/src/Data/Silver/2021 ( I Semestre).dta
Eliminado: /home/npalaciosv/Catedra/Geoanalitycs/src/Data/Silver/2019_SemII.sav
Eliminado: /home/npalaciosv/Catedra/Geoanalitycs/src/Data/Silver/2019_SemI.dta
Eliminado: /home/npalaciosv/Catedra/Geoanalitycs/src/Data/Silver/2018_SemI.sav
Eliminado: /home/npalaciosv/Catedra/Geoanalitycs/src/Data/Silver/2018_SemII.dta
Eliminado: /home/npalaciosv/Catedra/Geoanalitycs/src/Data/Silver/2020_SemI.dta
Eliminado: /home/npalaciosv/Catedra/Geoanalitycs/src/Data/Silver/2020_SemII.sav
Eliminado: /home/npalaciosv/Catedra/Geoanalitycs/src/Data/Silver/2019_SemI.sav
Eliminado: /home/npalaciosv/Catedra/Geoanalitycs/src/Data/Silver/2018_SemII.sav
Eliminado: /home/npalaciosv/Catedra/Geoanalitycs/src/Data/Silver/2021 ( I Semestre).sav
Eliminado: /home/npalaciosv/

⚠️ **Alertas:** Algunos archivos se descomprimen dentro de una carpeta, por lo tanto nos toca moverlos hacia el directorio Silver

In [5]:
for file in silver_files.iterdir():
    if file.is_file():
        pass
    else:
        subfolder = silver_files.joinpath(file.name)
        for subfile in subfolder.glob('*'):
            if subfile.suffix == '.csv':
                subfile.replace(silver_path+"/"+subfile.name)
                print(f'Archivo: {subfile.name} movido exitosamente')

        shutil.rmtree(subfolder)
        print(f'Carpeta: {subfolder.name} elminada')

Archivo: 2021(II Semestre).csv movido exitosamente
Carpeta: 2021 (II.semestre) elminada
Archivo: SIPSA_A_Isem2022.csv movido exitosamente
Carpeta: 2022 ( I Semestre ) elminada
Archivo: SIPSA_A Isem2024.csv movido exitosamente
Carpeta: 2024 ( I Semestre ) elminada
Archivo: SIPSA_A_IIsem2023.csv movido exitosamente
Carpeta: 2023 ( II Semestre ) elminada
Archivo: SIPSA_A_Isem2023.csv movido exitosamente
Carpeta: SIPSA_A_Isem2023 elminada


### **3. Unificamos la informacion que necesitamos**

➡️ Creamos un dataframe vacio y ahi concatenamos toda la informacion que necesitamos.

In [6]:
df = pd.DataFrame()

⚠️ **Alertas:** Es posible que las bases de datos no tengan exactamente las mismas columnas. Previamente preparamos un diccionario que nos va a ayudar a normalizar los nombres.

In [7]:
columnas_ajustadas = {
    'Fuente':'Mayorista',
    'FechaEncuesta':'Fecha',
    'Fecha':'Fecha',
    'Cod. Depto Proc.':'Codigo_departamento',
    'Cod. Municipio Proc.':'Codigo_municipio',
    'Departamento Proc.':'Departamento',
    'Municipio Proc.':'Municipio',
    'Grupo':'Grupo',
    'Ali':'Alimento',
    'Cant Kg':'Cant_Kg',
    'Código Departamento':'Codigo_departamento',
    ' Código Municipio ':'Codigo_municipio',
    'Alimento':'Alimento',
    'Cuidad, Mercado Mayorista':'Mayorista',
    'Origen':'Origen',
    'Unnamed: 9':'Unnamed: 9',
    'Unnamed: 10':'Unnamed: 10',
    'Codigo CPC':'Codigo CPC'
}

➡️ Ajustamos los nombres de todos los archivos de acuerdo al diccionario que acabamos de definir.

In [8]:
for file in silver_files.iterdir():
    df_temp = pd.read_csv(file, encoding='unicode_escape', sep=";")
    df_temp['Origen'] = file.name
    for column in df_temp.columns:
        df_temp.rename(columns={column: columnas_ajustadas[column]}, inplace=True)
    df =pd.concat([df,df_temp], ignore_index=True)

/tmp/ipykernel_272404/1997498870.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(file, encoding='unicode_escape', sep=";")


➡️ Eliminamos las columnas que no vamos a utilizar.

In [9]:
df = df.drop(['Unnamed: 9','Unnamed: 10','Codigo CPC'],axis=1)

➡️ Homologamos dentro de la columna *"Fuente"* a *"Cali, Santa Elena"* por *"Cali, Santa Helena"*



In [10]:
df.loc[df["Mayorista"] == "Cali, Santa Elena", "Mayorista"] = "Cali, Santa Helena"
df['Codigo_municipio'] = df['Codigo_municipio'].str.replace("'","")
df['Codigo_municipio'] = df['Codigo_municipio'].str.strip()
df['Codigo_departamento'] = df['Codigo_departamento'].str.replace("'","")
df['Codigo_departamento'] = df['Codigo_departamento'].str.strip()
df['Cant_Kg'] = pd.to_numeric(df['Cant_Kg'], errors='coerce')

➡️ Enviamos la informacion a un archivo csv para ver como se está juntando el resultado.

In [11]:
user_path = '/mnt/d/Classes/Data visualization/Mapping data/Database/geo.db'
conn = sqlite3.connect(user_path)
df.to_sql('Fact', con=conn, if_exists='replace', index=False)

11764957

### **4. Iniciamos con la construccion de los mapas**

#### **4.1 Establecemos las coordenadas de los centros mayoristas**

➡️ Primero importamos las coordenadas de cada uno de los centros de acopia que se mencionan en la base de datos

In [12]:
coordenadas_path = "/home/npalaciosv/Catedra/Geoanalitycs/src/Data/Static/Coordenadas.csv"
df_coor = pd.read_csv(coordenadas_path,sep=";",encoding='utf-8')

conn = sqlite3.connect(user_path)
df_coor.to_sql('Coordenadas_mayoristas', con=conn, if_exists='replace', index=False)

32

➡️ Creamos el objeto mapa utilizando la libreria folium. Especificamos un punto central para que aparezca el mapa.

In [13]:
map = folium.Map(location=[4.922860659232988, -74.02580517889908], zoom_start=7)

➡️ Llamamos el icono de una plaza de mercado que escogimos para ponerlo como icono de marcacion.

In [14]:
#
maps_path = '/home/npalaciosv/Catedra/Geoanalitycs/src/Graphs/'
icon_path = '/home/npalaciosv/Catedra/Geoanalitycs/src/Icons/Mayorista.png'

#
lat = df_coor['Latitud'].to_list()
lon = df_coor['Longitud'].to_list()
fuente = df_coor['Mayorista'].to_list()
coordenadas = list(zip(lat,lon,fuente))

#
for lat,lon,fuente in coordenadas:
    folium.Marker(
        [lat,lon],
        popup=f'{fuente}',
        icon=folium.CustomIcon(icon_image=icon_path,icon_size=(25,25))
    ).add_to(map)
map.save(maps_path+"mapa.html")

#### **4.2 Establecemos las coordenadas de los municipios abastecedor**

Cargamos la informacion que aparece en el archvio Clasificador_geografico.csv

In [15]:
# Definir tipos de datos específicos para las columnas
column_types = {'Código Departamento': str, 'Código Municipio': str, 'Código Centro Poblado': str}

geopath = '/home/npalaciosv/Catedra/Geoanalitycs/src/Data/Static/Clasificador_geografico.csv'
df_geoclas = pd.read_csv(geopath,sep=";",encoding='utf-8', dtype=column_types)

# Crear un diccionario para mapear los nombres de las columnas
column_rename_map = {
    'Código Departamento': 'Codigo_Departamento',
    'Código Municipio': 'Codigo_Municipio',
    'Código Centro Poblado': 'Codigo_Centro_Poblado'
}

# Renombrar las columnas usando el método rename
df_geoclas = df_geoclas.rename(columns=column_rename_map)

In [16]:
df_geoclas = df_geoclas[['Codigo_Departamento','Codigo_Municipio','Longitud','Latitud']]
df_geoclas = df_geoclas.drop_duplicates(subset=['Codigo_Municipio'])

df_geoclas['Latitud'] = df_geoclas['Latitud'].str.replace(",",".")
df_geoclas['Longitud'] = df_geoclas['Longitud'].str.replace(",",".")

coordenadas_int_path = "/home/npalaciosv/Catedra/Geoanalitycs/src/Data/Static/Coordenadas_internacionales.csv"
df_coor_int = pd.read_csv(coordenadas_int_path,sep=";",encoding='utf-8')

# Convertir las columnas 'Latitud' y 'Longitud' a tipo float (double en pandas)
df_coor_int['Latitud'] = df_coor_int['Latitud'].astype(float)
df_coor_int['Longitud'] = df_coor_int['Longitud'].astype(float)

list_coor_int = df_coor_int['Municipio'].to_list()

Vamos a crear una sentencia SQL que nos traiga las ubicaciones de los proveedores que necesitamos

In [17]:
conn = sqlite3.connect(user_path)
df_geoclas.to_sql('Clasificador_geografico', con=conn, if_exists='replace', index=False)

1122

In [18]:
conn = sqlite3.connect(user_path)
cursor = conn.cursor()
sentencia_sql = f'''
                SELECT
                    DISTINCT f.Codigo_Municipio,
                    f.Municipio,
                    cg.Latitud,
                    cg.Longitud
                FROM Fact f
                LEFT JOIN Clasificador_geografico cg ON cg.Codigo_Municipio = f.Codigo_Municipio
                    ;
                '''

cursor.execute(sentencia_sql)

# Obtener los resultados de la consulta
results = cursor.fetchall()

In [19]:
# Mostrar los resultados
df_int = pd.DataFrame(results, columns=['Codigo_Municipio', 'Municipio', 'Latitud', 'Longitud'])

for mun_int in list_coor_int:
    df_int.loc[df_int['Municipio']==mun_int,'Latitud'] = df_coor_int.loc[df_coor_int['Municipio']==mun_int,'Latitud'].values
    df_int.loc[df_int['Municipio']==mun_int,'Longitud'] = df_coor_int.loc[df_coor_int['Municipio']==mun_int,'Longitud'].values

df_int = df_int.dropna()

conn = sqlite3.connect(user_path)
df_int.to_sql('Coordenadas_internacionales', con=conn, if_exists='replace', index=False)

1147

In [20]:
#
icon_path = '/home/npalaciosv/Catedra/Geoanalitycs/src/Icons/Productor.png'

#
lat = df_int['Latitud'].to_list()
lon = df_int['Longitud'].to_list()
municipio = df_int['Municipio'].to_list()
coordenadas = list(zip(lat,lon,municipio))

#
for lat,lon,municipio in coordenadas:
    folium.Marker(
        [lat,lon],
        popup=f'{municipio}',
        icon=folium.CustomIcon(icon_image=icon_path,icon_size=(15,15))
    ).add_to(map)
map.save(maps_path+"mapa.html")

#### **4.3 Coloreamos cada departamento de acuerdo a su capacidad productora**

In [21]:
geopath = '/home/npalaciosv/Catedra/Geoanalitycs/src/Data/Static/colombia.geo.json'

folium.GeoJson(geopath,
               style_function= lambda feature : {
                        'color': 'black',
                        'weight': 2,
                        'dashArray': '5, 5',
                        'fillOpacity': 0.5,
                        }
               ).add_to(map)
map.save(maps_path+"mapa.html")

In [22]:
conn = sqlite3.connect(user_path)
cursor = conn.cursor()
sentencia_sql = f'''
                SELECT
                    f.Codigo_departamento AS DPTO,
                    SUM(f.Cant_Kg) AS Produccion
                FROM Fact f
                WHERE f.Codigo_departamento != 'n.a.'
                GROUP BY f.Codigo_departamento
                ORDER BY Produccion DESC
                ;
                '''

cursor.execute(sentencia_sql)

# Obtener los resultados de la consulta
results = cursor.fetchall()

In [23]:
# Ejecutar la consulta y cargar los resultados en un DataFrame
df_temp = pd.DataFrame(results, columns=['DPTO', 'Produccion'])

# Convertir el DataFrame a un diccionario
temp_dict = df_temp.to_dict(orient='records')
temp_dict

[{'DPTO': '25', 'Produccion': 6786107919.296},
 {'DPTO': '15', 'Produccion': 4400427211.527},
 {'DPTO': '05', 'Produccion': 3804643763.755},
 {'DPTO': '68', 'Produccion': 2730609255.394},
 {'DPTO': '76', 'Produccion': 2481002743.13},
 {'DPTO': '52', 'Produccion': 2423965237.539},
 {'DPTO': '50', 'Produccion': 2326006609.249},
 {'DPTO': '54', 'Produccion': 1492876634.912},
 {'DPTO': '11', 'Produccion': 1481709028.603},
 {'DPTO': '73', 'Produccion': 1290401140.915},
 {'DPTO': '63', 'Produccion': 872649277.478},
 {'DPTO': '23', 'Produccion': 820047330.802},
 {'DPTO': '41', 'Produccion': 774962367.911},
 {'DPTO': '17', 'Produccion': 708300483.243},
 {'DPTO': '81', 'Produccion': 526311244.624},
 {'DPTO': '08', 'Produccion': 510248712.027},
 {'DPTO': '47', 'Produccion': 452783227.959},
 {'DPTO': '19', 'Produccion': 450456382.505},
 {'DPTO': '66', 'Produccion': 412911803.823},
 {'DPTO': '13', 'Produccion': 293710118.93},
 {'DPTO': '85', 'Produccion': 261060042.626},
 {'DPTO': '20', 'Produccio

In [24]:
# Función para asignar color basado en la producción
def asignar_color(feature):

    # Calcular el mínimo y máximo de las producciones
    producciones = [item['Produccion'] for item in temp_dict]
    produccion_minima = min(producciones)
    produccion_maxima = max(producciones)*0.5

    # Definir colores en la escala
    color_minimo = 'red'
    color_medio = 'orange'
    color_maximo = 'green'

    codigo_departamento = feature['properties'].get('DPTO')
    print(codigo_departamento)
    produccion = next((item['Produccion'] for item in temp_dict if item['DPTO'] == codigo_departamento), 0)  # Obtener la producción o 0 si no está definido

    # Normalizar la producción entre 0 y 1
    norm = mcolors.Normalize(vmin=produccion_minima, vmax=produccion_maxima)

    # Crear una interpolación lineal de colores
    color_interp = mcolors.LinearSegmentedColormap.from_list('custom_map', [color_minimo, color_medio, color_maximo])

    # Obtener el color según la producción normalizada
    color = mcolors.rgb2hex(color_interp(norm(produccion)))

    return color

In [25]:
folium.GeoJson(geopath,
               style_function= lambda feature : {
                        'fillColor':asignar_color(feature),
                        'color': 'black',
                        'weight': 2,
                        'dashArray': '5, 5',
                        'fillOpacity': 0.25,
                        }
               ).add_to(map)
map.save(maps_path+"mapa.html")

05
05
08
11
13
15
17
18
19
20
23
25
27
41
44
47
50
52
54
63
66
68
70
73
76
81
85
86
91
94
95
97
99
88


#### **4.3 Vinculamos los origenes productivos con las plazas mayoristas destino**

In [26]:
conn = sqlite3.connect(user_path)
cursor = conn.cursor()
sentencia_sql = f'''
                SELECT
                    f.Mayorista,
                    cm.Latitud AS Latitud_may,
                    cm.Longitud AS Longitud_may,
                    f.Municipio,
                    COALESCE(cg.Latitud, ci.Latitud) AS Latitud_mun,
                    COALESCE(cg.Longitud, ci.Longitud) AS Longitud_mun,
                    SUM(f.Cant_Kg) AS Produccion
                FROM Fact f
                LEFT JOIN Coordenadas_mayoristas cm ON cm.Mayorista = f.Mayorista
                LEFT JOIN Clasificador_geografico cg ON cg.Codigo_municipio = f.Codigo_municipio
                LEFT JOIN Coordenadas_internacionales ci ON ci.Municipio = f.Municipio
                GROUP BY f.Mayorista,cm.Latitud,cm.Longitud,f.Municipio,cg.Latitud,cg.Longitud
                ORDER BY Produccion DESC
                ;
                '''

cursor.execute(sentencia_sql)

# Obtener los resultados de la consulta
results = cursor.fetchall()

# Ejecutar la consulta y cargar los resultados en un DataFrame
df_temp = pd.DataFrame(results, columns=['Mayorista', 'Latitud_may','Longitud_may','Municipio','Latitud_mun','Longitud_mun','Produccion'])
df_temp = df_temp.dropna()
df_temp

,Mayorista,Latitud_may,Longitud_may,Municipio,Latitud_mun,Longitud_mun,Produccion
0,"Bogotá, D.C., Corabastos",4.632496,-74.161960,GRANADA,3.5466952856,-73.7057158517,2.458141e+09
1,"Medellín, Central Mayorista de Antioquia",6.189480,-75.589830,LA UNIÓN,5.97387113417,-75.3608622911,6.963371e+08
2,"Bogotá, D.C., Corabastos",4.632496,-74.161960,MOSQUERA,4.70701024824,-74.2193416046,6.165378e+08
3,"Bogotá, D.C., Corabastos",4.632496,-74.161960,ZIPAQUIRÁ,5.02547142431,-73.9944757558,5.468878e+08
4,"Bogotá, D.C., Corabastos",4.632496,-74.161960,AQUITANIA,5.51860567807,-72.8839843241,5.168797e+08
...,...,...,...,...,...,...,...
9638,"Sincelejo, Nuevo Mercado",9.295541,-75.387066,PUERTO BERRÍO,6.48703921007,-74.4099459786,2.000000e+00
9639,"Bogotá, D.C., Paloquemao",4.615986,-74.084526,TIBASOSA,5.74722853114,-72.9994496769,1.800000e+00
9640,"Bogotá, D.C., Plaza Samper Mendoza",4.618221,-74.082652,SASAIMA,4.96216463559,-74.4326283935,1.500000e+00
9641,"Popayán, Plaza de mercado del barrio Bolívar",2.447850,-76.602805,SAN PEDRO DE CARTAGO,1.55148084166,-77.1194170341,1.200000e+00
